In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import joblib

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("../Data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis = "columns", how = "all")
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
X = df.drop("koi_disposition", axis = 1)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
y = df["koi_disposition"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100, stratify = y)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
5347,1,0,0,0,1.775553,4.290000e-05,-4.290000e-05,131.920000,0.023800,-0.023800,...,-219,4.390,0.060,-0.060,1.138,0.633,-0.123,295.43936,40.147449,13.335
3118,0,0,0,0,6.055520,6.370000e-05,-6.370000e-05,131.598510,0.009110,-0.009110,...,-159,4.585,0.024,-0.128,0.805,0.149,-0.064,295.47382,39.124561,14.772
6944,0,0,0,0,31.445271,1.698000e-04,-1.698000e-04,154.581360,0.005130,-0.005130,...,-86,4.198,0.130,-0.130,1.471,0.301,-0.247,297.47711,39.877239,14.806
20,0,0,0,0,4.996780,1.390000e-05,-1.390000e-05,171.742080,0.002190,-0.002190,...,-159,4.552,0.034,-0.136,0.847,0.161,-0.069,297.80826,47.946671,15.861
5831,1,0,0,0,3.671661,1.450000e-07,-1.450000e-07,132.090917,0.000032,-0.000032,...,-311,4.020,0.273,-0.147,1.940,0.520,-0.635,300.63577,44.053398,12.814


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [6]:
# Scale X values
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Label Encode the target feature from human words to machine id's
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [8]:
print(X_train_scaled.shape, y_train_categorical.shape)

(5243, 40) (5243, 3)


# Tune Model Parameters

In [9]:
# Create the model and the layers
model = Sequential()
model.add(Dense(units = 50, activation = "relu", input_dim = X_train_scaled.shape[1]))
model.add(Dense(units = 100, activation = "relu"))
model.add(Dense(units = 150, activation = "relu"))
model.add(Dense(units = 200, activation = "relu"))
model.add(Dense(units = y_train_categorical.shape[1], activation = "softmax"))

In [10]:
# Compile the model
model.compile(optimizer = "adam",
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2050      
                                                                 
 dense_1 (Dense)             (None, 100)               5100      
                                                                 
 dense_2 (Dense)             (None, 150)               15150     
                                                                 
 dense_3 (Dense)             (None, 200)               30200     
                                                                 
 dense_4 (Dense)             (None, 3)                 603       
                                                                 
Total params: 53,103
Trainable params: 53,103
Non-trainable params: 0
_________________________________________________________________


# Train Tuned Model

In [11]:
model.fit(X_train_scaled,
          y_train_categorical,
          epochs = 500,
          shuffle = True,
          verbose = 2,
          callbacks = None,
          class_weight = None,
          sample_weight = None,
          initial_epoch = 0,
          steps_per_epoch = None,
          validation_steps = None,
          validation_freq = 0.001,
          max_queue_size = 1000,
          workers = 4,
          use_multiprocessing = False)

Epoch 1/500
164/164 - 0s - loss: 0.4480 - accuracy: 0.7751 - 432ms/epoch - 3ms/step
Epoch 2/500
164/164 - 0s - loss: 0.3583 - accuracy: 0.8163 - 145ms/epoch - 884us/step
Epoch 3/500
164/164 - 0s - loss: 0.3434 - accuracy: 0.8362 - 150ms/epoch - 917us/step
Epoch 4/500
164/164 - 0s - loss: 0.3309 - accuracy: 0.8385 - 149ms/epoch - 906us/step
Epoch 5/500
164/164 - 0s - loss: 0.3191 - accuracy: 0.8541 - 177ms/epoch - 1ms/step
Epoch 6/500
164/164 - 0s - loss: 0.3319 - accuracy: 0.8425 - 149ms/epoch - 908us/step
Epoch 7/500
164/164 - 0s - loss: 0.3106 - accuracy: 0.8566 - 146ms/epoch - 892us/step
Epoch 8/500
164/164 - 0s - loss: 0.3073 - accuracy: 0.8585 - 151ms/epoch - 921us/step
Epoch 9/500
164/164 - 0s - loss: 0.3031 - accuracy: 0.8640 - 169ms/epoch - 1ms/step
Epoch 10/500
164/164 - 0s - loss: 0.2896 - accuracy: 0.8692 - 155ms/epoch - 943us/step
Epoch 11/500
164/164 - 0s - loss: 0.2856 - accuracy: 0.8764 - 158ms/epoch - 964us/step
Epoch 12/500
164/164 - 0s - loss: 0.2827 - accuracy: 0.873

# Quantify our Trained Model

In [12]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose = 2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 1.1550 - accuracy: 0.8432 - 162ms/epoch - 3ms/step
Normal Neural Network - Loss: 1.1550114154815674, Accuracy: 0.8432494401931763


# Save the Model

In [13]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
model.save("../Models/Deep_Learning_Model.h5")